In [2]:
import pandas as pd
import numpy as np
import re
from datetime import datetime

In [3]:
data=pd.read_csv('../small_business/data/restaurants.csv')
data=data.drop(columns='Unnamed: 0')

## FUNCTIONS 

In [4]:
def keep_numeric(x):
    return re.sub("[^0-9]", "", x)

def replace_price(x):
    if x == "£":
        return 1
    if x == "££": 
        return 2 
    if x == "£££": 
        return 3 
    if x == "££££": 
        return 4 
    
#Function latitude/ longitude

def latitude(column):
    key = pd.read_csv("../raw_data/api_key.csv", header=None)
    key = key.loc[0,0]
    gmaps = googlemaps.Client(key=key)
    # Geocoding an address
    geocode_result = gmaps.geocode(column)
    return float(geocode_result[0]['geometry']['location']['lat'])

def longitude(column):
    key = pd.read_csv("../raw_data/api_key.csv", header=None)
    key = key.loc[0,0]
    gmaps = googlemaps.Client(key=key)
    # Geocoding an address
    geocode_result = gmaps.geocode(column)
    return float(geocode_result[0]['geometry']['location']['lng'])

def get_postal_code(x):
    g = re.findall(r'\d{4}-\d{3}', x)
    if g ==[]: 
        return '1100-083'
    else: 
        return g[0]
    
#Function labels 
def labels_eat_on_site (x):
    if 'dine-in' in x:
        return 1 
    else: 
        return 0
    
def labels_delivery (x):
    if ('delivery' in x) or ('no-contact delivery' in x):
        if ( 'no delivery' in (x)):
            return 0
        else: 
            return 1
    else: 
        return 0
    
def labels_takeaway (x):
    if ('takeaway' in x):
        return 1
    else: 
        return 0

def labels_drive_thru (x):
    if ('drive-through' in x):
        return 1
    else: 
        return 0
    
def labels_no_del(x):
    if ( 'no delivery' in (x)):
        return 1
    else: 
        return 0    
    
def labels_curbside(x):
    if ( 'curbside pickup' in (x)):
        return 1
    else: 
        return 0
    
#post code format for post codes table 
def post_code2(x):
    return (x[0:4]+'-'+x[4:])  

In [5]:
def keep_numeric(x):
    return re.sub("[^0-9]", "", x)

In [6]:
def replace_price(x):
    if x == "£":
        return 1
    if x == "££": 
        return 2 
    if x == "£££": 
        return 3 
    if x == "££££": 
        return 4 

In [7]:
#Function latitude/ longitude

def latitude(column):
    key = pd.read_csv("../raw_data/api_key.csv", header=None)
    key = key.loc[0,0]
    gmaps = googlemaps.Client(key=key)
    # Geocoding an address
    geocode_result = gmaps.geocode(column)
    return float(geocode_result[0]['geometry']['location']['lat'])

def longitude(column):
    key = pd.read_csv("../raw_data/api_key.csv", header=None)
    key = key.loc[0,0]
    gmaps = googlemaps.Client(key=key)
    # Geocoding an address
    geocode_result = gmaps.geocode(column)
    return float(geocode_result[0]['geometry']['location']['lng'])

In [8]:
def get_postal_code(x):
    g = re.findall(r'\d{4}-\d{3}', x)
    if g ==[]: 
        return '1100-083'
    else: 
        return g[0]

In [9]:
#Function labels 
def labels_eat_on_site (x):
    if 'dine-in' in x:
        return 1 
    else: 
        return 0
    
def labels_delivery (x):
    if ('delivery' in x) or ('no-contact delivery' in x):
        if ( 'no delivery' in (x)):
            return 0
        else: 
            return 1
    else: 
        return 0
    
def labels_takeaway (x):
    if ('takeaway' in x):
        return 1
    else: 
        return 0

def labels_drive_thru (x):
    if ('drive-through' in x):
        return 1
    else: 
        return 0
    
def labels_no_del(x):
    if ( 'no delivery' in (x)):
        return 1
    else: 
        return 0    
    
def labels_curbside(x):
    if ( 'curbside pickup' in (x)):
        return 1
    else: 
        return 0

In [10]:
#post code format for post codes table 
def post_code2(x):
    return (x[0:4]+'-'+x[4:])  

In [11]:
def deletespace(x):
    return x.strip()

def replace_type(x):
    result=x
    for k, v in dico.items():
        if x in v:
            result=k
    return result

## Preprocessing of columns

In [361]:
# Importing the data in form 1 and basic preprocessing 
#data= pd.read_csv("../raw_data/restaurants2 - Copy.csv", header= None)
#data=data.rename(columns={0:"Index", 1:"name" , 2:"type", 3:"description", 4:"rating", 5:"review_count", 6: "price", 7:"address", 8:"label"})
#data=data.drop(columns='Index') # Check whether to keep it or not 

In [378]:
# Importing the data in form 2 and basic preprocessing (this takes a bit of time)
data= pd.read_csv("../raw_data/restaurants_clean (2).csv")
data=data.rename(columns={0:"Index", 'restaurant_name':"name" , 'restaurant_category':"type", 'restaurant_description':"description", 'restaurant_rating':"rating", 'restaurant_comment_number':"review_count", 'restaurant_price_range': "price", 'restaurant_location':"address", 'restaurant_services':"label"})
data=data.drop(columns=['Index', 'Unnamed: 0']) # Check whether to keep it or not 
data=data.dropna(subset=['rating']) # dropping rows with no rating 
data.label=data.label.fillna(value='Dine-in')# replacing rows with no label by ['Dine-in']
data.type=data.type.fillna(value='Restaurant')

In [379]:
# Column rating 
data.rating=pd.to_numeric(data.rating, downcast="float")

# Column review_count 
data.review_count=data.review_count.map(keep_numeric)
data.review_count=pd.to_numeric(data.review_count, downcast="float")

#Column price 
data.price = data.price.map(replace_price)

#Adress into latitude and longitude 
#data['latitude']=data.address.map(latitude)
#data['longitude']=data.address.map(longitude)

# handling the label 
data.label= data['label'].str.lower()
data['dine_in']=data.label.map(labels_eat_on_site)
data['takeaway']=data.label.map(labels_takeaway)
data['delivery']=data.label.map(labels_delivery)
data['drive_through']=data.label.map(labels_drive_thru)
data['no_del_exp']=data.label.map(labels_no_del)
data['curb_pickup']=data.label.map(labels_curbside)

# handling the type
data.type=data.type.replace('restaurant', '', regex=True)
data.type= data.type.str.lower()

#add postal code
data['postal_code']=data.address.map(get_postal_code)

#handling remaining null values (price):
#data.price=data.price.fillna(value=np.mean(data.price)).astype(int) ## CHOICE 1 TO BE CONFIRMED: FILLING THE NA O


In [364]:
data.head(2)

,name,type,description,rating,review_count,price,address,label,dine_in,takeaway,delivery,drive_through,no_del_exp,curb_pickup,postal_code
0,Augusto Lisboa,brunch,"Hip brunch eatery serving savory toasts, banan...",4.8,1032.0,2.0,"Rua Santa M.nha 26, 1100-491 Lisboa, Portugal","['dine-in', 'takeaway', 'no delivery']",1,1,0,0,1,0,1100-491
1,Tiffin Cafe & Restaurant Lisboa,brunch,NaN,4.9,139.0,2.0,"R. do Conde 32, 1200-637 Lisboa, Portugal","['dine-in', 'takeaway', 'delivery']",1,1,1,0,0,0,1200-637


## Prepropressing of postal codes table and merging with original database

In [380]:
data_post=pd.read_csv("../raw_data/cod_post_freg_matched.txt")
data_post.CodigoPostal=data_post.CodigoPostal.astype(str)
data_post.CodigoPostal=data_post.CodigoPostal.map(post_code2)
data_post=data_post[['CodigoPostal', 'Concelho', 'Freguesia']]
data_post=data_post.rename(columns={'CodigoPostal':'postal_code', 'Distrito': 'district', 'Concelho': 'municipality', 'Freguesia': 'neighborhood'} )
data=data.merge(data_post, on='postal_code', how='left')

In [318]:
data.head()

,name,type,description,rating,review_count,price,address,label,dine_in,takeaway,delivery,drive_through,no_del_exp,curb_pickup,postal_code,municipality,neighborhood
0,Augusto Lisboa,brunch,"Hip brunch eatery serving savory toasts, banan...",4.8,1032.0,2.0,"Rua Santa M.nha 26, 1100-491 Lisboa, Portugal","['dine-in', 'takeaway', 'no delivery']",1,1,0,0,1,0,1100-491,Lisboa,Graça
1,Tiffin Cafe & Restaurant Lisboa,brunch,NaN,4.9,139.0,2.0,"R. do Conde 32, 1200-637 Lisboa, Portugal","['dine-in', 'takeaway', 'delivery']",1,1,1,0,0,0,1200-637,Lisboa,Prazeres
2,Crisfama,restaurant,NaN,4.8,649.0,1.0,"Rua da Cruz de Santa Apolónia 56, 1100-188 Lis...","['dine-in', 'takeaway', 'no delivery']",1,1,0,0,1,0,1100-188,Lisboa,Santa Engrácia
3,Alma,fine dining,High-end restaurant with avant-garde local mea...,4.8,953.0,4.0,"R. Anchieta 15, 1200-224 Lisboa, Portugal","['dine-in', 'no takeaway', 'no delivery']",1,1,0,0,1,0,1200-224,Lisboa,São Paulo
4,Restaurant CHULHO & Bar,restaurant,NaN,4.9,180.0,NaN,"R. de São João da Mata 82, 1200-850 Lisboa, Po...","['dine-in', 'takeaway', 'delivery']",1,1,1,0,0,0,1200-850,Lisboa,Lapa


In [319]:
#data.to_csv('../raw_data/Clean_data_1_12_v2.csv')